In [1]:
import pandas as pd
from datetime import datetime, timedelta
import getpass  # To get the current system user's name

# -----------------------------
# STEP 1: CREATE SAMPLE INPUT DATA
# -----------------------------

# Master data simulating metadata for EOL Group configurations
master_df = pd.DataFrame({
    'Version': ['V1', 'V1'],
    'Version Name': ['Ver1', 'Ver1'],
    'Location': ['Loc1', 'Loc1'],
    'EOL Group ID': ['EG1', 'EG1'],
    'Sub Location': ['SLoc1', 'SLoc1'],
    'Item': ['Item1', 'Item2'],
    'EOL Group Item Site ASN': [1, 0],                  # Only 'Item1' should be valid (ASN=1)
    'EOL Group Detail_Delete Flag': [False, False]      # Only include where Delete Flag is False
})

# Detail input containing EOL constraint details per item
detail_df = pd.DataFrame({
    'Version': ['V1', 'V1'],
    'Version Name': ['Ver1', 'Ver1'],
    'Location': ['Loc1', 'Loc1'],
    'EOL Group ID': ['EG1', 'EG1'],
    'Sub Location': ['SLoc1', 'SLoc1'],
    'Item': ['Item1', 'Item2'],

    # EOP constraint date range
    'EOL Group Const EOP Min Date': [pd.Timestamp('2025-07-20'), pd.Timestamp('2025-07-10')],
    'EOL Group Const EOP Max Date': [pd.Timestamp('2025-07-30'), pd.Timestamp('2025-07-15')],

    # Apply flag indicates if this constraint should be applied
    'EOL Group Const Apply Flag': [True, True],

    # Constraint period range
    'EOL Group Const Constraint Start Date': [pd.Timestamp('2025-07-01'), pd.Timestamp('2025-07-01')],
    'EOL Group Const Constraint End Date': [pd.Timestamp('2025-08-01'), pd.Timestamp('2025-08-01')],
    'EOL Group Const Constraint Qty': [100, 200],

    # Expected constraint values
    'EOL Group Const Demand Qty': [50, 150],
    'EOL Group Const Demand Short Qty': [10, 20],
    'EOL Group Const Prod Result Qty': [60, 160],
    'EOL Group Const Prod Plan Qty': [70, 170],
    'EOL Group Const Pegging Inv Qty': [30, 130],

    # Metadata for constraint record
    'EOL Group Const Creation time': [pd.Timestamp.now(), pd.Timestamp.now()],
    'EOL Group Const Created by': [getpass.getuser(), getpass.getuser()],

    # Actual EOP Dates (used for comparison with constraint min/max)
    'Master_EOP_Plan_Date': [pd.Timestamp('2025-07-25'), pd.Timestamp('2025-07-12')],
    'Master_EOP_Change_Date': [pd.Timestamp('2025-07-27'), pd.Timestamp('2025-07-14')],
})

# Demand input contains current and short demand for items
demand_df = pd.DataFrame({
    'Version': ['V1', 'V1'],
    'Location': ['Loc1', 'Loc1'],
    'Item': ['Item1', 'Item2'],
    'EOL Demand_Demand Qty': [100, 200],
    'EOL Demand_Demand Short Qty': [20, 30]
})

# Plan input contains production and inventory results for items
plan_df = pd.DataFrame({
    'Version': ['V1', 'V1'],
    'Location': ['Loc1', 'Loc1'],
    'Sub Location': ['SLoc1', 'SLoc1'],
    'Item': ['Item1', 'Item2'],
    'EOL Plan Result_Prod Result Qty': [100, 200],
    'EOL Plan Result_Prod Plan Qty': [100, 200],
    'EOL Plan Result_Pegging Inv Qty': [50, 150]
})

In [2]:
# -----------------------------
# STEP 2: FILTER MASTER BY FLAGS
# -----------------------------

# Filter out rows where 'EOL Group Item Site ASN' != 1 or 'Delete Flag' is True
# This ensures we're only working with valid EOL group items
filtered_master = master_df[
    (master_df['EOL Group Item Site ASN'] == 1) & 
    (~master_df['EOL Group Detail_Delete Flag'])
]

In [3]:
# -----------------------------
# STEP 3: MERGE DETAIL WITH DEMAND & PLAN DATA
# -----------------------------

# First merge: combine detail with demand data on Version, Location, and Item
detail_with_demand = detail_df.merge(
    demand_df,
    on=['Version', 'Location', 'Item'],
    how='left'
)

# Second merge: combine with plan data on Version, Location, Sub Location, and Item
# This results in a comprehensive detail DataFrame with demand and plan metrics
detail_full = detail_with_demand.merge(
    plan_df,
    on=['Version', 'Location', 'Sub Location', 'Item'],
    how='left'
)


In [4]:
# -----------------------------
# STEP 4: APPLY RULES / CONSTRAINT LOGIC
# -----------------------------

# Calculate min and max of actual EOP dates (plan vs change) per row
actual_min_date = detail_full[['Master_EOP_Plan_Date', 'Master_EOP_Change_Date']].min(axis=1)
actual_max_date = detail_full[['Master_EOP_Plan_Date', 'Master_EOP_Change_Date']].max(axis=1)

# Build individual boolean masks for each business rule

# 1. Constraint min date should be <= actual min of EOP dates
mask_min_date = detail_full['EOL Group Const EOP Min Date'] <= actual_min_date

# 2. Constraint max date should be <= actual max of EOP dates
mask_max_date = detail_full['EOL Group Const EOP Max Date'] <= actual_max_date

# 3. Apply flag must be True
mask_apply_flag = detail_full['EOL Group Const Apply Flag'].astype(bool)

# 4. Demand Qty constraint should be less than or equal to actual demand
mask_demand_qty = detail_full['EOL Group Const Demand Qty'] <= detail_full['EOL Demand_Demand Qty']

# 5. Demand short Qty constraint should be <= actual short demand
mask_short_qty = detail_full['EOL Group Const Demand Short Qty'] <= detail_full['EOL Demand_Demand Short Qty']

# 6. Production result must satisfy constraint
mask_prod_result = detail_full['EOL Group Const Prod Result Qty'] <= detail_full['EOL Plan Result_Prod Result Qty']

# 7. Production plan Qty must satisfy constraint
mask_prod_plan = detail_full['EOL Group Const Prod Plan Qty'] <= detail_full['EOL Plan Result_Prod Plan Qty']

# 8. Pegging inventory Qty must satisfy constraint
mask_inv_qty = detail_full['EOL Group Const Pegging Inv Qty'] <= detail_full['EOL Plan Result_Pegging Inv Qty']

# 9. Creation time must be <= now + 9 hours (future check allowed)
mask_time = detail_full['EOL Group Const Creation time'] <= (pd.Timestamp.now() + pd.Timedelta(hours=9))

# 10. Created by must match current system user
mask_user = detail_full['EOL Group Const Created by'] == getpass.getuser()

# Combine all masks into a single filter using logical AND
final_mask = (
    mask_min_date & mask_max_date & mask_apply_flag &
    mask_demand_qty & mask_short_qty &
    mask_prod_result & mask_prod_plan & mask_inv_qty &
    mask_time & mask_user
)

# Filter the detail data with all constraints applied
filtered_detail = detail_full[final_mask]

# -----------------------------
# STEP 5: HANDLE "NO MATCH" CASE
# -----------------------------

# If no items pass the constraints, default to updating all detail rows
if filtered_detail.empty:
    print("⚠️ No matching input. Applying to ALL detail records.")
    filtered_detail = detail_full.copy()
else:
    print("✅ Filtered detail rows found based on constraints.")

# -----------------------------
# STEP 6: SHOW FINAL OUTPUT
# -----------------------------

# Display key results
print("\n🔍 Final Processed Data:")
print(filtered_detail[['Version', 'Item', 'EOL Group Const Demand Qty', 'EOL Demand_Demand Qty']])

⚠️ No matching input. Applying to ALL detail records.

🔍 Final Processed Data:
  Version   Item  EOL Group Const Demand Qty  EOL Demand_Demand Qty
0      V1  Item1                          50                    100
1      V1  Item2                         150                    200
